<a href="https://colab.research.google.com/github/CS-Edwards/games_and_friends/blob/main/Part_2_OpenAI_Function_Calling_weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Games and Friends 😊🎲

Love board games? In this demo we build a workflow with Weaviate (Python v.4) and OpenAI to query our database suggests games to play and friends to play them with using natural language prompts.

This demo is split into two parts (and two notebooks):

-  Part 1: Weaviate Client and Generative Search
-  *Part 2: Open AI Function Calling

**THIS NOTEBOOK IS PART TWO.**

In this notebook we will:

1. Connect to the Weaviate client database from Part 1
2. Create functions to update and query the database
3. Establish OpenAI Function Calling
4. Demo games and friends


In [2]:
!pip install -q -U weaviate-client # Python v.4
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!

In [86]:
import weaviate
import openai
import pandas as pd
import json

from weaviate.classes.query import MetadataQuery
from typing import Dict, Any, List, Optional, Tuple, Union



In [85]:
#@title ▶️ Load Local Utility Functions

def execute_route_function(conv_result: any) -> any:
    """
    Execute a route function based on the provided conversation result.

    Args:
        conv_result (any): The conversation result containing information about the function to execute.

    Returns:
        any: The result of executing the route function.

    Extracts the name and arguments of the route function from the conversation result and then
    dynamically executes the function using the extracted information.

    Note:
        - The function_name and function_args are extracted from the tool_calls attribute of the conversation result.
        - The function_name is used to retrieve the actual function from the global namespace.
        - The function_args are passed as arguments to the retrieved function.
    """

    function_name = conv_result.tool_calls[0].function.name
    function_args = json.loads(conv_result.tool_calls[0].function.arguments)
    function = globals()[function_name]
    return function(**function_args)


def print_game(game_result: List[Any]) -> None:
    """
    Print the title and description of each game in the game result.

    Args:
        game_result (List[Any]): List of objects representing games.
            Each object should have properties 'title' and 'description'.
    """
    for game_res in game_result:
        # Print game title
        print(game_res.properties['title'])

        # Print game description
        print(game_res.properties['description'])


def print_player(player_result: List[Any]) -> None:
    """
    Print the name of each player in the player result.

    Args:
        player_result (List[Any]): List of objects representing players.
            Each object should have a property 'name'.
    """
    for player_res in player_result:
        # Print player name
        print(player_res.properties['name'])



def output_route(result: Union[Dict[str, Any], str]) -> None:
    """
    Print the appropriate output based on the type of result.

    Args:
        result (Union[Dict[str, Any], str]): The result to be processed.
            If it's a dictionary, it's assumed to contain 'generated_resp' and 'player_result' keys.
            If it's a string, (a uuid from either inserting a game or player) print success message.
    """
    if isinstance(result, dict):
        print("It's game time! We recommend:")
        print(result['generated_resp'])
        print("*************************")
        print("We think your friend(s)...")
        print_player(result['player_result'])
        print("would love to play this with you!")
    else:
        print("You've successfully updated the database")


### CLIENT: Weaviate Cloud Database

In [5]:

#
# Set environment variables
#

# For Colab "Secrets"
from google.colab import userdata

URL = userdata.get("WCS_URL")
APIKEY = userdata.get("WCS_API_KEY")
OPENAIKEY = userdata.get("OPENAI")


In [6]:
#
# Connect to WCS instance
#

with weaviate.connect_to_wcs(
    cluster_url=URL,
    auth_credentials=weaviate.auth.AuthApiKey(APIKEY),
    headers={
        "X-OpenAI-Api-Key": OPENAIKEY
    }
  )as client:
    print(f'Weaviate Client Ready: {client.is_ready()}')
    print(f'Weaviate Client Connected: {client.is_connected()}')

Weaviate Client Ready: True
Weaviate Client Connected: True


In [23]:
#
# Access collections
#
client.connect()
game = client.collections.get("Game")
player = client.collections.get("Player")

### Function Summaries:

- `insert_player():` Add player to database.
- `find_player():` Query players from the database.
- `insert_game():`  Add games to the database
- `find_game():` Retrieve game from database and generate response
- `find_players_and_games():` Controller function that first calls find_game, and then called find_player (s) based on the game result

In [9]:
#@title ▶️ insert_player()


#
# Add a player
#


def insert_player(name: str, profile: str, favoriteGame: Optional[str] = None, age: int = 30 )-> str:
    '''
    Insert player data into a database.

    Args:
        name (str): The name of the player.
        favoriteGame (str, optional): The favorite game of the player. Defaults to None.
        age (int, optional): The age of the player. Defaults to 30.
        profile (str): The player's profile.

    Raises:
        ValueError: If `name` or `profile` is not provided.
    '''
    if not name or not profile:
        raise ValueError("Name and profile must be provided.")

    return player.data.insert(
          properties={
              "name": name,
              "age": age,
              "favoriteGame": favoriteGame,
              "profile": profile
          }
      )



In [10]:
# test_player_overton_uuid = insert_player(
#     name="Overton",
#     favoriteGame="Chess",
#     age=25,
#     profile="Avid chess player with a passion for strategy."
# )


In [11]:
# print(type(test_player_overton_uuid))

In [12]:
# player.query.fetch_object_by_id(test_player_overton_uuid)

In [13]:
#@title ▶️ find_player()


#
# Find (query) player/s
#

def find_players(game_obj) -> List:
    '''
    Find players based on game properties.

    Args:
        game_props (Dict[str, Any]): Dictionary containing game properties.

    Returns:
        List: List containing response objects.
    '''
    response = player.query.near_text(
        query=f"{game_obj}",
        limit=int(game_obj.properties['bestPlayers']),
        return_metadata=MetadataQuery(distance=True)
    )

    response_list = []
    for o in response.objects:
        response_list.append(o)

    return response_list





In [14]:

# players_list = find_players(game_result)
# for player_obj in players_list:
#     print(player_obj.properties)
#     print(player_obj.metadata.distance)

In [15]:
#@title ▶️ insert_game()

#
# Add game
#

def insert_game(title: str, description: str, subtitle: str = None,
                primaryType: Optional[str] = None, secondaryType: Optional[str] = None,
                minPlayers: int = 1, maxPlayers: int = 3, bestPlayers: int = 2,
                minPlayTime: int = 10, minAge: int = 12)->str:
    '''
    Insert game data into a database.

    Args:
        title (str): The title of the game.
        description (str): The description of the game.
        subtitle (str): The subtitle of the game. Defaults to None.
        primaryType (str, optional): The primary type of the game. Defaults to None.
        secondaryType (str, optional): The secondary type of the game. Defaults to None.
        minPlayers (int): The minimum number of players for the game. Defaults to 1.
        maxPlayers (int): The maximum number of players for the game. Defaults to 3.
        bestPlayers (int): The recommended number of players for the game. Defaults to 2.
        minPlayTime (int): The minimum playtime for the game in minutes. Defaults to 10.
        minAge (int): The minimum age required to play the game. Defaults to 12.

    Raises:
        ValueError: If any required argument is not provided.
    '''
    if not all((title, description)):
        raise ValueError("Title, and description must be provided.")

    return game.data.insert(
        properties={
            "title": title,
            "subtitle": subtitle,
            "primaryType": primaryType,
            "secondaryType": secondaryType,
            "description": description,
            "minPlayers": minPlayers,
            "maxPlayers": maxPlayers,
            "bestPlayers": bestPlayers,
            "minPlayTime": minPlayTime,
            "minAge": minAge
        }
    )


In [16]:
# test_game_uno_uuid = insert_game(
#     title="Uno",
#     subtitle="The classic card game",
#     description="UNO is a multi-player card game in which the objective is to be the \
#                 player to get rid of all the cards in their hand. Each player is dealt \
#                 7 cards and players take turn drawing cards from the deck.",
#     primaryType="Card Game",
#     minPlayers=2,
#     maxPlayers=10,
#     bestPlayers=4,
#     minPlayTime=30,
#     minAge=6
# )

# print(test_game_uno_uuid)

In [44]:
# game_result = game.query.fetch_object_by_id(test_game_uno_uuid)
# print(game_result.properties['bestPlayers'])

In [71]:
#@title ▶️ find_games()

#
#  Find (query) game/s
#

def find_games(query: str, limit: int = 1) -> List:
    '''
    Find games based on a user query.

    Args:
        query (str): The query to search for games.
        limit (int, optional): Maximum number of games to return. Defaults to 1.

    Returns:
        List: List containing response objects.
        Str: Generated response based on prompt.
    '''
    # response = game.query.near_text(
    #     query=query,
    #     limit=limit,
    #     return_metadata=MetadataQuery(distance=True)
    # )
    generate_prompt = "Tell me why this is a suitable game based on what I am looking for, and why this game is so much fun, include a brief summary of the game"

    response = game.generate.near_text(
        query= query,
        grouped_task=generate_prompt,
        limit=limit
    )

    response_list = []
    for o in response.objects:
        response_list.append(o)

    return response_list, response.generated  # Return the list containing the response objects, and generated response


In [30]:
# find_game_result = find_games("Fun family game with mystery", 2)
# print(find_game_result)


In [18]:

# for game_res in find_game_result:
#   print(game_res.properties['title'])

In [32]:
#@title ▶️ find_players_and_games()

def find_players_and_games(query: str) -> Dict[str, Tuple[List[str], List[str]]]:
    """
    Find games based on the query and then find players for the first game.

    Args:
        query (str): The query string to search for games.

    Returns:
        dict: A dictionary containing game results and player results.
            Keys:
                - "generated_response": Response from generator.
                - "player_result": A tuple containing a list of players found for the first game.
    """
    game_result, generated_resp = find_games(query)
    player_result = find_players(game_result[0])

    return {"generated_resp": generated_resp, "player_result": player_result}


In [72]:
player_game_result = find_players_and_games("family card game")


In [74]:
player_game_result

{'generated_resp': 'Clank!: Legacy – Acquisitions Incorporated is a suitable game for someone looking for a unique and immersive gaming experience. This game is based on the legacy format, where the game permanently changes over a series of sessions, introducing new rules and mechanics as the campaign progresses. \n\nIn this game, players take on the role of adventurers exploring a dungeon filled with treasures and dangers. The game incorporates elements of deck-building and strategy as players race to acquire the most valuable loot while avoiding traps and monsters. The legacy aspect of the game adds a new layer of excitement as players make permanent changes to the game board and components based on their choices and outcomes.\n\nThe game is not only fun because of its engaging gameplay mechanics, but also because of the storytelling aspect that evolves as the campaign unfolds. Players will feel a sense of accomplishment as they see the game world change and adapt to their decisions.

In [42]:
#print_player(player_game_result["player_result"])


## Open AI Function Calling

Open AI function calling is a tool that uses large language models (LLMs) to suggests a function and parameters that should be called based on the input query and the description(s) of the the function.

For our use case we have added the `insert_game()`, `insert_player`, and `find_players_and_games` functions into the `tools` array of the `route_query` function below. In the `tools` array we provide the name, description and type of each function and their respective parameters. `route_query` uses `gpt 3.5`, to identify the most relevant function for a given natural langauge input, and extracts the parameters from the input.

IMPORTANT NOTE: OpenAI function calling **DOES NOT** actually call/execute the function.

We execute the functions suggested by function calling using a local utility method called : `execute_route_function()`


**Resources**:
- Open AI Function Calling: [Documentation](https://platform.openai.com/docs/guides/function-calling)

In [47]:
#
# Connect to Open AI Client
#
#



from openai import OpenAI
import os
import json

os.environ["OPENAI_API_KEY"] = OPENAIKEY
client = OpenAI()



In [45]:
#
# Open AI Function call setup with gpt-3.5-turbo-0125
#


def route_query(user_query:str)->str:
    """
      Route user queries to either add games/players to Weaviate database or find (query) players and games based on a user query, providing recommendations for games and players likely to enjoy them.

      Args:
          query (str): The user's input query.

      Returns:
          Dict[str, List]: A dictionary containing the game result and player result.

      This function initiates a conversation with OpenAI's language model, passing the user query and available functions.

      Step 1: Send the conversation and available functions to the model.
          - Each function is described with its name, description, and parameters.
          - Three functions are available:
              1. insert_player: For queries relating to adding players into the database.
                Parameters include name, profile, favorite game, and age.
              2. insert_game: For queries related to adding a game into the database.
                Parameters include title, description, subtitle, primary type, secondary type,
                min players, max players, best players, min playtime, and min age.
              3. find_players_and_games: For queries requesting a recommendation for a game to play with friends.
                The function returns a game based on the user's request and players that will likely enjoy the game.

      Returns the response from the language model, specifically the first choice message from the available choices.
      """


    messages = [{"role": "user", "content": user_query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "insert_player",
                "description": "For queries relating to adding players into database",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the player",
                        },

                        "profile":{
                            "type": "string",
                            "description":"About me, description of the player"

                        },

                        "favoriteGame":{
                            "type":"string",
                            "description":"The favorite game of the player. Defaults to None",
                        },

                        "age":{
                            "type":"integer",
                            "description":"The age of the player. Defaults to 30"
                        }
                    },
                    "required": ["name","profile"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "insert_game",
                "description": "For queries related to adding a game into the database.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "The title of the game",
                        },
                        "description": {
                            "type": "string",
                            "description": "Description of the game",
                        },
                        "subtitle": {
                            "type": "string",
                            "description": "Tagline of the game. Defaults to None",
                        },
                        "primaryType": {
                            "type": "string",
                            "description": "The primary type of the game (ie. mystery, fantasy, strategy). Defaults to None",
                        },
                        "secondaryType": {
                            "type": "string",
                            "description": "The secondary type of the game (ie. mystery, fantasy, strategy). Defaults to None",
                        },
                        "minPlayers": {
                            "type": "integer",
                            "description": "The minimum number of players for the game. Defaults to 1.",
                        },
                        "maxPlayers": {
                            "type": "integer",
                            "description": "The maximum number of players for the game. Defaults to 3.",
                        },
                        "bestPlayers": {
                            "type": "integer",
                            "description": "The recommended number of players for the game. Defaults to 2.",
                        },
                        "minPlayTime": {
                            "type": "integer",
                            "description": "The minimum playtime for the game in minutes. Defaults to 10.",
                        },
                        "minAge": {
                            "type": "integer",
                            "description": "The minimum age required to play the game. Defaults to 12.",
                        },
                    },
                    "required": ["title","description"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "find_players_and_games",
                "description": "For queries requesting a recommendation for a game to play with friends, the function will return a game based on the users request and players that will likely enjoy the game ",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Input user query",
                        }
                    },
                    "required": ["query"],
                },
            },
        }

    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default
    )
    return response.choices[0].message


In [48]:
#
# Test 1: Prompt to add new player to the database
#
# Expected Function: 'insert_player'
# Expected Paramters: name = "Kyle", age = 25, favoriteGame = Poker, profile = " Enjoys managing risk and making decision based on skill and analysis of incomplete information " (or something similar)
#


query_1 = "Kyle is a 25 year old stock broker his favorite game is Poker because it requires managing risk and making decision based on skill and analysis of incomplete information"
function_call_result_1 = route_query(query_1)


In [49]:
for tool_call in function_call_result_1.tool_calls:
    if tool_call.function:
        arguments_dict = eval(tool_call.function.arguments)
        function_name = tool_call.function.name
        arguments_str = ", ".join([f"{key}={value}" for key, value in arguments_dict.items()])
        print(f"function called: {function_name}() \nparameters: {arguments_str}")

function called: insert_player() 
parameters: name=Kyle, profile=Kyle is a 25-year-old stock broker who enjoys games that require managing risk and making decisions based on skill and analysis of incomplete information., favoriteGame=Poker, age=25


In [50]:
#
# Test 2: Prompt to add new game to the database
#
# Expected Function: 'insert_game'
# Required Paramters: title = "Spades", description = "trick-taking card game where players bid on the number of tricks they can win each round and aim to fulfill their bid while preventing opponents from doing the same.
# It requires strategic thinking, teamwork (in partnership variations), and careful card counting to outwit opponents and successfully fulfill bids. (or something similar)
#


query_2 = "Add a new game: Spades is a classic trick-taking card game where players bid on the number of tricks they can win each round and aim to fulfill their \
bid while preventing opponents from doing the same. It requires strategic thinking, teamwork (in partnership variations), and careful card counting \
to outwit opponents and successfully fulfill bids."
function_call_result_2 = route_query(query_2)

In [ ]:
function_call_result_2

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2qt9rzW3Ka0Vern6JHGZIj9f', function=Function(arguments='{"title":"Spades","description":"Spades is a classic trick-taking card game where players bid on the number of tricks they can win each round and aim to fulfill their bid while preventing opponents from doing the same. It requires strategic thinking, teamwork (in partnership variations), and careful card counting to outwit opponents and successfully fulfill bids.","primaryType":"Card Game","minPlayers":4,"maxPlayers":4,"bestPlayers":4,"minPlayTime":30,"minAge":12}', name='insert_game'), type='function')])

In [ ]:
for tool_call in function_call_result_2.tool_calls:
    if tool_call.function:
        arguments_dict = eval(tool_call.function.arguments)
        function_name = tool_call.function.name
        arguments_str = ", ".join([f"{key}={value}" for key, value in arguments_dict.items()])
        print(f"function called: {function_name}() \nparameters: {arguments_str}")

function called: insert_game() 
parameters: title=Spades, description=Spades is a classic trick-taking card game where players bid on the number of tricks they can win each round and aim to fulfill their bid while preventing opponents from doing the same. It requires strategic thinking, teamwork (in partnership variations), and careful card counting to outwit opponents and successfully fulfill bids., primaryType=Card Game, minPlayers=4, maxPlayers=4, bestPlayers=4, minPlayTime=30, minAge=12


In [75]:
#
# Test 3: Prompt to find a game to play and people to play with
#
# Expected Function: 'find_players_and_games'

query_3 = "An fantasy adventure game for a group of people"
function_call_result_3 = route_query(query_3)

In [76]:
for tool_call in function_call_result_3.tool_calls:
    if tool_call.function:
        arguments_dict = eval(tool_call.function.arguments)
        function_name = tool_call.function.name
        arguments_str = ", ".join([f"{key}={value}" for key, value in arguments_dict.items()])
        print(f"function called: {function_name}() \nparameters: {arguments_str}")

function called: find_players_and_games() 
parameters: query=I am looking for a fantasy adventure game for a group of people


In [77]:
#result_test = execute_route_function(function_call_result_3)


In [79]:
#print(result_test)

In [ ]:
#
# Execute code
#

In [51]:
# Test the function with the provided example
result = execute_route_function(function_call_result_2)
print(result)

51adbea0-1321-4043-b25e-d742e00a7a00


In [52]:
print(type(result))

<class 'uuid.UUID'>


In [54]:
result2 = execute_route_function(function_call_result_3)
print(type(result2))


<class 'dict'>


In [55]:
result2

{'generated_resp': "The Lord of the Rings: Journeys in Middle-Earth is a suitable game for you if you are looking for a cooperative and strategic card game set in the fantasy world of Middle-earth. This game allows players to work together against the forces of Sauron, rather than competing against each other. The deck construction aspect of the game adds depth and customization, making each playthrough unique.\n\nThe game is a lot of fun because it immerses players in the rich world of J.R.R. Tolkien's literary works, with thematic content, artwork, and challenges that are replayable by different approaches. The cooperative gameplay and player interaction make it especially engaging for those who want to explore Middle-earth with friends.\n\nIn The Lord of the Rings: Journeys in Middle-Earth, players take on the roles of heroes exploring Middle-earth and combating evil in an epic campaign. With a variety of quests, adventures, and characters, the game offers a dynamic and immersive ex

## Demo Games and Friends 😊🎲

🎉We have successfully completed our Games and Friends workflow!

The `games_and_friends_controller` function manages the flow of our program and handles any errors that may arise.

Let the games begin!

In [82]:
def games_and_friends_controller(query: str) -> None:
    """
    Controller function for managing the execution flow of the games and friends program.

    This function routes the query to the appropriate function, executes it, and outputs the result.

    Args:
        query (str): The query string.
    """
    try:
        function_call_result = route_query(query)

        executed_function_result = execute_route_function(function_call_result)

        output_route(executed_function_result)

    except Exception as e:
        print("An error occurred:", e)
        print("Please try again, with a different prompt/query.")


In [84]:
#
# Input your query below
#

query = input("Enter your Games and Friends query: ")


Enter your Games and Friends query: A fantasy adventure game for a group of people


In [80]:
#query = "A fantasy adventure game for a group of people"

In [83]:
#
# Run to see your results :)
#


games_and_friends_controller(query)

It's game time! We recommend:
The Lord of the Rings: Journeys in Middle-Earth is a suitable game for you if you are looking for a cooperative and strategic card game set in the fantasy world of Middle-earth. This game allows players to work together against the forces of Sauron, rather than competing against each other. The deck construction aspect of the game adds depth and customization, making it engaging for players who enjoy strategic gameplay.

The game is a lot of fun because it immerses players in the rich world of Middle-earth, with thematic content, artwork, and challenges that are replayable in different ways. The cooperative nature of the game encourages teamwork and communication among players, creating a sense of camaraderie as they work together to overcome obstacles.

In The Lord of the Rings: Journeys in Middle-Earth, players take on the roles of heroes exploring Middle-earth and embarking on an epic campaign to combat evil. With a variety of quests, adventures, and ch